Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
from dotenv import load_dotenv

In [50]:
def get_last_message(thread_id):
    message = client.beta.threads.messages.list(thread_id=thread_id)
    message_data = message.data[0]

    message_id = message_data.id
    assistant_id = message_data.assistant_id
    created_at = message_data.created_at
    file_ids = message_data.file_ids
    role = message_data.role
    run_id = message_data.run_id
    message_text = message_data.content[0].text.value

    dic_message = {'message_id': message_id, 'assistant_id': assistant_id, 
                     'created_at': created_at, 'file_ids': file_ids, 'role': role, 'run_id': run_id,
                     'message_text': message_text}
    
    return dic_message

In [2]:
## configruation

# load_dotenv('config.env')
# OPEN_API_KEY = os.getenv('OPEN_API_KEY')

# List of JSON files
# json_files = [('income_stmt', 'df_income_stmt_columns.json'), ('cashflow', 'df_cashflow_columns.json'), ('stocks', 'df_stocks_columns.json')]

# Load schemas from JSON file
with open('dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

with open('config.json', 'r') as f:
    dic_config = json.load(f)
    OPEN_API_KEY = dic_config['OPEN_API_KEY']
    FIN_ANALYST_ID = dic_config['fin_analyst_assistant']

# Initialize an empty list to store the stocks
stocks_list = []

# Open the text file in read mode
with open('stocks.txt', 'r') as file:
    # Read each line of the file
    for line in file:
        # Strip any leading or trailing whitespace and append the line to the list
        stocks_list.append(line.strip())

df_portfolio = pd.read_csv('portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

client = OpenAI(api_key=OPEN_API_KEY)

In [7]:
client.files.list()

SyncPage[FileObject](data=[FileObject(id='file-uaNv0CqIo4bQFbOfNeB1yPYB', bytes=43214, created_at=1710835073, filename='df_income_stmt.csv', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-Hqyh5j4mqhIbAYDMJGc7Mftd', bytes=16433312, created_at=1710833965, filename='df_stocks.csv', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-Vb86kSr9GkcgcFXVXc1tgFLJ', bytes=22633, created_at=1710786690, filename='AWS Transfer Family for unified file delivery (2024-03-14 11_30 GMT-7) - Transcript.txt', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-RQ2Xs7wiOI5eZY6lnMSiRliA', bytes=42448, created_at=1710530778, filename='Remove CSID Constraint (2024-03-15 09_05 GMT-7) - Transcript.txt', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-gMaWufkcQcf9TgDZEgvjI7do', bytes=4300, created_at=1709738822, filena

### Stocks Data

In [3]:
stocks_list = ['MSFT', 'SHOP', 'AMD', 'NVDA', 'AAPL', 'OKTA', 'SHOP', 'AMAT', 'TTWO', 'SNOW', 'SOXX', 'BA', 'DOCN', 'MCD', 'LULU', 'CSCO', 'ORCL', 'AMZN', 'ASML']

stocks_str = ' '.join(stocks_list)

stocks = yf.Tickers(stocks_str)

In [40]:
# How to set period parameter:
# week = wk
# month = mo
# year = y

# Create an empty df_stocks dataframe
df_stocks = pd.DataFrame(columns=dict(schemas['stocks']))

# Create an empty df_cashflow dataframe
df_cashflow = pd.DataFrame(columns=dict(schemas['cashflow']))

# Create an empty df_income_stmt dataframe
df_income_stmt = pd.DataFrame(columns=dict(schemas['income_stmt']))

# Loop through each stock
for stock in stocks_list:

    # Append to df_stocks
    df_plc = stocks.tickers[stock].history(period="max").reset_index()
    df_plc['Ticker'] = stock

    df_stocks = pd.concat([df_stocks, df_plc])

    # Append to df_cashflow
    df_plc = stocks.tickers[stock].cashflow.T.reset_index()
    df_plc.rename(columns={'index': 'Date'},inplace=True)
    df_plc['Ticker'] = stock

    df_cashflow = pd.concat([df_cashflow, df_plc])

    # Append to df_income_stmt
    df_plc = stocks.tickers[stock].income_stmt.T.reset_index()
    df_plc.rename(columns={'index': 'Date'},inplace=True)
    df_plc['Ticker'] = stock

    df_income_stmt = pd.concat([df_income_stmt, df_plc])

# Write to CSV
df_stocks.to_csv('openai_upload_files/df_stocks.csv', index=False)
df_cashflow.to_csv('openai_upload_files/df_cashflow.csv', index=False)
df_income_stmt.to_csv('openai_upload_files/df_income_stmt.csv', index=False)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_29484\4059506789.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_stocks = pd.concat([df_stocks, df_plc])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_29484\4059506789.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the 

In [65]:
dic_files = {}

for key in dic_files:
    try:
        client.files.delete(dic_files[key])
    except:
        print('file id: ' + dic_files[key] + ' already deleted')

for filename in os.listdir('openai_upload_files'):

    # Upload a file with an "assistants" purpose
    openai_file = client.files.create(
        file=open(f'openai_upload_files/{filename}', "rb"),
        purpose='assistants'
    )
    dic_files[filename] = openai_file.id

with open('openai_files.json', 'w') as json_file:
    json.dump(dic_files, json_file)

In [66]:
for key in dic_files:
    try:
        client.files.delete(dic_files[key])
    except:
        print('file id: ' + dic_files[key] + ' already deleted')
# for filename in os.listdir('openai_upload_files'):
#     print(filename)

In [135]:
# Assistant id asst_I4lFSi5mtU5OYoeivxOc4piG

client.beta.assistants.update(assistant_id=fin_analyst.id, instructions='You are a financial assistant. Your task is to analyze the stock market and provide an analysis on the market or performance of specific stocks.\
         The objective is to identify whether the specific stocks would make a good investment for a 6 months period. Use the income statement knowledge base you have.')
# file_stocks.id

Assistant(id='asst_I4lFSi5mtU5OYoeivxOc4piG', created_at=1708929718, description=None, file_ids=['file-gCVBXhVyflB0SvusXy4793Kz'], instructions='You are a financial assistant. Your task is to analyze the stock market and provide an analysis on the market or performance of specific stocks.         The objective is to identify whether the specific stocks would make a good investment for a 6 months period. Use the income statement knowledge base you have.', metadata={}, model='gpt-4-turbo-preview', name='Financial Assistant', object='assistant', tools=[ToolRetrieval(type='retrieval')])

In [40]:
# I think financial analyst is too broad
# Need to divide the work. Maybe one RAG studies the financial statements
# One model evaluates the risk based on trends and sentiments

fin_analyst = client.beta.assistants.create(
    name='Financial Assistant',
    instructions='You are a financial assistant. Your task is to analyze the stock market and provide an analysis on the market or performance of specific stocks.\
         The objective is to identify whether the specific stocks would make a good investment for a 6 months period.',
    model='gpt-4-turbo-preview',
    tools=[{'type': 'retrieval'}]
)

In [15]:
# Set dictionary for tracking messages
dic_messages = {}

## Create thread
thread = client.beta.threads.create(
    messages=[
        {
            'role': 'user',
            'content': 'Can you tell me what you are and what do you do?'
        }
    ]
)

# Get thread id
thread_id = thread.id

# Record first message
dic_messages['0'] = get_last_message(thread_id)

# Run assistants
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id=FIN_ANALYST_ID
)

run_id = run.id

run_retrieve = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run_id
)

run_status = run_retrieve.status

if run_status == 'compledted':
    pass
elif run_status == 'cancelled':
    pass
elif run_status == 'failed':
    pass
elif run_status == 'expired':
    pass

# final step is to delete the thread
# client.beta.threads.delete(thread_id=THREAD_ID)

In [21]:
# client.beta.threads.retrieve(thread_id=thread.id)
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id=FIN_ANALYST_ID
)

# run_retrieve = client.beta.threads.runs.retrieve(
#   thread_id=thread.id,
#   run_id=run.id
# )

run_retrieve.status

# messages = client.beta.threads.messages.list(
#   thread_id=thread.id
# )
# client.beta.threads.delete(thread_id=thread.id)

'completed'

In [ ]:
SyncCursorPage[ThreadMessage](
    data=[ThreadMessage(id='msg_8LAgBdoruiq7qExZkbXyWlq4', assistant_id='asst_I4lFSi5mtU5OYoeivxOc4piG', content=[MessageContentText(text=Text(annotations=[], value="I am an AI developed by OpenAI, designed to assist you with a wide range of tasks. My capabilities include processing and analyzing text-based information, providing explanations, summaries, or insights on various topics, answering questions, and offering guidance or advice within the constraints of my programming. Specifically, in this context, I can help analyze financial documents, interpret data, and provide an analysis of the stock market or the performance of specific stocks based on the information available in documents you've provided or based on general finance principles. My goal is to support you in making informed decisions or gaining a better understanding of complex subjects."), type='text')], created_at=1711229792, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_TFbiUQc8B52YFXGsuQ3E7aPn', thread_id='thread_Vfn67f8h7LIwzba52KDpYFFk'), 
          ThreadMessage(id='msg_6dDKDzhRD09Ksjb0TNMHdJB9', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Can you tell me what you are and what do you do?'), type='text')], created_at=1711229119, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_Vfn67f8h7LIwzba52KDpYFFk')], 
          object='list', 
          first_id='msg_8LAgBdoruiq7qExZkbXyWlq4', 
          last_id='msg_6dDKDzhRD09Ksjb0TNMHdJB9', 
          has_more=False)

In [44]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="This is another test message, please ignore"
    # file_ids=[file_income_stmt_test.id]
)

ThreadMessage(id='msg_Bp1EXNpq7dv2jAZa7TpCqS8S', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='This is another test message, please ignore'), type='text')], created_at=1711434209, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_FGDsn9Jx8j5SjpgDKUZbo7ql')

In [137]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=fin_analyst.id
)

In [ ]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [147]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [172]:
print(messages.data[0].content[0].text.value)

Based on the financial data provided in the document for Microsoft Corporation (MSFT) as of June 30, 2023, here is an analysis that may help in evaluating its stock for a potential investment over the next 6 months:

1. **Strong Financial Performance**: Microsoft has shown a robust financial performance. Key highlights include:
   - **Total Revenue**: $211.915 billion, indicating a strong top-line growth.
   - **Gross Profit**: $146.052 billion, demonstrating a healthy margin that reflects efficient management and strong pricing power.
   - **Net Income**: $72.361 billion, illustrating a solid bottom line that provides room for future investments and returns to shareholders.

2. **Healthy EBITDA and EPS**: Microsoft reported an EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization) of $105.140 billion and a normalized EBITDA of $105.155 billion, suggesting strong operational effectiveness. The Diluted Earnings Per Share (EPS) is $9.68, and the Basic EPS is $9.72, both

In [112]:
# get historical market data
# hist = msft.history(period="1mo")
stocks.tickers['SHOP'].income_stmt.T.reset_index().dtypes
# df = stocks.tickers['SHOP'].cashflow.T.reset_index()
# df.rename(columns={'index': 'Date'},inplace=True)
# df

index                                                         datetime64[ns]
Tax Effect Of Unusual Items                                           object
Tax Rate For Calcs                                                    object
Normalized EBITDA                                                     object
Total Unusual Items                                                   object
Total Unusual Items Excluding Goodwill                                object
Net Income From Continuing Operation Net Minority Interest            object
Reconciled Depreciation                                               object
Reconciled Cost Of Revenue                                            object
EBITDA                                                                object
EBIT                                                                  object
Net Interest Income                                                   object
Interest Expense                                                      object